In [1]:
import numpy as np
import sys
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.layers import LSTM
from keras.utils import np_utils
import utils
from keras.preprocessing.sequence import pad_sequences

# Performs classification using LSTM network.

FREQ_DIST_FILE = r'C:\Users\Yash\training1-freqdist.pkl'
BI_FREQ_DIST_FILE = r'C:\Users\Yash\training1-freqdist-bi.pkl'
TRAIN_PROCESSED_FILE = r'C:\Users\Yash\training1-processed.csv'
TEST_PROCESSED_FILE = r'C:\Users\Yash\training1-processed.csv'
GLOVE_FILE = r'C:\Users\Yash\gs50d.txt'
dim = 50


def get_glove_vectors(vocab):
    print ('Looking for GLOVE vectors')
    glove_vectors = {}
    found = 0
    with open(GLOVE_FILE, 'r', encoding='utf8') as glove_file:
        for i, line in enumerate(glove_file):
            utils.write_status(i + 1, 0)
            tokens = line.split()
            word = tokens[0]
            if vocab.get(word):
                vector = [float(e) for e in tokens[1:]]
                glove_vectors[word] = np.array(vector)
                found += 1
    print ('\n')
    print ('Found %d words in GLOVE' % found)
    return glove_vectors


def get_feature_vector(tweet):
    words = tweet.split()
    feature_vector = []
    for i in range(len(words) - 1):
        word = words[i]
        if vocab.get(word) is not None:
            feature_vector.append(vocab.get(word))
    if len(words) >= 1:
        if vocab.get(words[-1]) is not None:
            feature_vector.append(vocab.get(words[-1]))
    return feature_vector


def process_tweets(csv_file, test_file=True):
    tweets = []
    labels = []
    print ('Generating feature vectors')
    with open(csv_file, 'r') as csv:
        lines = csv.readlines()
        total = len(lines)
        for i, line in enumerate(lines):
            if test_file:
                tweet_id, tweet = line.split(',')
            else:
                tweet_id, sentiment, tweet = line.split(',')
            feature_vector = get_feature_vector(tweet)
            if test_file:
                tweets.append(feature_vector)
            else:
                tweets.append(feature_vector)
                labels.append(int(sentiment))
            utils.write_status(i + 1, total)
    print ('\n')
    lbl=np.array(labels)
    label= np_utils.to_categorical(lbl, num_classes=3)
    return tweets, label


if __name__ == '__main__':
    train = 1
    np.random.seed(1337)
    vocab_size = 90000
    batch_size = 500
    max_length = 40
    filters = 600
    kernel_size = 3
    vocab = utils.top_n_words(FREQ_DIST_FILE, vocab_size, shift=1)
    glove_vectors = get_glove_vectors(vocab)
    tweets, labels = process_tweets(TRAIN_PROCESSED_FILE, test_file=False)
    embedding_matrix = np.random.randn(vocab_size + 1, dim) * 0.01
    for word, i in vocab.items():
        glove_vector = glove_vectors.get(word)
        if glove_vector is not None:
            embedding_matrix[i] = glove_vector
    tweets = pad_sequences(tweets, maxlen=max_length, padding='post')
    shuffled_indices = np.random.permutation(tweets.shape[0])
    tweets = tweets[shuffled_indices]
    labels = labels[shuffled_indices]
    if train:
        model = Sequential()
        model.add(Embedding(vocab_size + 1, dim, weights=[embedding_matrix], input_length=max_length))
        model.add(Dropout(0.4))
        model.add(LSTM(128))
        model.add(Dense(64))
        model.add(Dropout(0.5))
        model.add(Activation('relu'))
        model.add(Dense(3))
        model.add(Activation('sigmoid'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        filepath = r"C:\Users\Yash\models3\lstm-{epoch:02d}-{loss:0.3f}-{acc:0.3f}-{val_loss:0.3f}-{val_acc:0.3f}.hdf5"
        checkpoint = ModelCheckpoint(filepath, monitor="loss", verbose=1, save_best_only=True, mode='min')
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=0.000001)
        print (model.summary())
        model.fit(tweets, labels, batch_size=128, epochs=5, validation_split=0.1, shuffle=True, callbacks=[checkpoint, reduce_lr])
    else:
        model = load_model(sys.argv[1])
        print (model.summary())
        test_tweets, _ = process_tweets(TEST_PROCESSED_FILE, test_file=True)
        test_tweets = pad_sequences(test_tweets, maxlen=max_length, padding='post')
        predictions = model.predict(test_tweets, batch_size=128, verbose=1)
        results = zip(map(str, range(len(test_tweets))), np.round(predictions[:, 0]).astype(int))
        utils.save_results_to_csv(results, 'lstm.csv')


Using TensorFlow backend.


Looking for GLOVE vectors
Processing 1193514/0ocessing 13593/0Processing 15593/0Processing 31792/0Processing 67590/0Processing 132787/0Processing 134787/0Processing 140987/0Processing 177585/0Processing 179585/0Processing 183585/0Processing 185585/0Processing 197784/0Processing 201984/0Processing 212183/0Processing 216183/0Processing 228582/0Processing 270379/0Processing 354573/0Processing 356573/0Processing 366772/0Processing 368772/0Processing 370772/0Processing 372772/0Processing 407770/0Processing 409770/0Processing 458368/0Processing 479367/0Processing 547962/0Processing 622555/0Processing 624955/0Processing 656351/0Processing 686948/0Processing 708946/0Processing 718544/0Processing 885124/0Processing 971516/0Processing 977715/0Processing 1065711/0Processing 1125907/0Processing 1138306/0Processing 1165904/0Processing 1191303/0

Found 9970 words in GLOVE
Generating feature vectors
Processing 7976/7976

_________________________________________________________________
Layer (type)  